In [1]:
import sys
sys.path.append('../src')

In [2]:
import torch
from transformer_wrappers.wrappers.context import ContextCausalLMWrapper
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM

In [3]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda')

In [4]:
COLOURS = [f'C{i}' for i in range(10)]
STYLES = ['solid', 'dotted', 'dashed', 'dashdot']

In [5]:
import os
TOKEN = os.environ['HUGGINGFACE_TOKEN']

In [6]:
MODEL = 'gpt2-xl'
# MODEL = 'mistralai/Mistral-7B-Instruct-v0.2'  
# MODEL = 'meta-llama/Llama-2-7b-hf'
# MODEL = 'google/gemma-7b'
# MODEL = 'EleutherAI/pythia-14m'
MODEL_CONFIGS = {
    'torch_dtype': torch.bfloat16,
    'attn_implementation': 'eager',
    'device_map': 'cpu',
    'token': TOKEN,
}

TOKENIZER = 'gpt2-xl'
# TOKENIZER = 'mistralai/Mistral-7B-Instruct-v0.2'  
# TOKENIZER = 'meta-llama/Llama-2-7b-hf'
#TOKENIZER = 'google/gemma-7b'
#TOKENIZER = 'EleutherAI/pythia-14m'

TOKENIZER_CONFIGS = {'token': TOKEN}

In [7]:
model = ContextCausalLMWrapper.from_pretrained(MODEL, 
                                               att_len=2,
                                               model_kwargs=MODEL_CONFIGS, 
                                               tokenizer_name_or_path=TOKENIZER, 
                                               tokenizer_kwargs=TOKENIZER_CONFIGS,
                                               )
model.enable_benchmarking()

/home/nicolobrunello/miniconda/envs/nlp/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
model.transformer_wrapper.att_len


2

In [9]:
input_string = 'The capital of Italy is'

In [10]:
input_encodings = model.tokenizer(input_string, return_tensors='pt').to('cpu')

In [11]:
output_wrapper = model(
    **input_encodings,
    return_dict=True,
    output_attentions=True,
    use_cache=True,
    output_hidden_stsates=True,
    return_attention_output=True,  # Self-attention layer output
    return_feed_forward_output=True
)

/home/nicolobrunello/Documents/Projects/transformer_wrappers/notebooks/../src/transformer_wrappers/wrappers/context.py:217: UserWarning: Use of index_put_ on expanded tensors is deprecated. Please clone() the tensor before performing this operation. This also applies to advanced indexing e.g. tensor[indices] = tensor (Triggered internally at ../aten/src/ATen/native/TensorAdvancedIndexing.cpp:713.)
  attention_mask[..., mask] = torch.finfo(current_hidden_state.dtype).min
/home/nicolobrunello/Documents/Projects/transformer_wrappers/notebooks/../src/transformer_wrappers/wrappers/context.py:217: UserWarning: Use of masked_fill_ on expanded tensors is deprecated. Please clone() the tensor before performing this operation. This also applies to advanced indexing e.g. tensor[mask] = scalar (Triggered internally at ../aten/src/ATen/native/TensorAdvancedIndexing.cpp:1912.)
  attention_mask[..., mask] = torch.finfo(current_hidden_state.dtype).min


In [12]:
output = model.generate(input_encodings.input_ids,
                        do_sample=False, 
                        max_length=25, 
                        return_dict_in_generate=True, 
                        output_attentions=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [13]:
model.tokenizer.decode(output['sequences'].squeeze())

'The capital of Italy is is is is is is is is is is is is is is is is is is is is is'

In [15]:
output.attentions[2][0][0, 0, 0, :]

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.4824, 0.5195],
       dtype=torch.bfloat16)